# Using API and Basic Python for Data Analysis

## Introduction

In this exercise we will be pulling some data from the Quandl API. Qaundl is currently the most widely used aggregator of financial market data.

We will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called [Carl Zeiss Meditec](https://www.zeiss.com/meditec/int/home.html), which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery. The company is listed under the stock ticker AFX_X.

**Note:** we will only be using the Python standard library along with requests, json and numpy

## API Key
In order to run the code, please insert your own API Key below. You will have to sign up and log in to [quandl](http://www.quandl.com/).

In [95]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ' '

## Import Modules and Initial Exploration

In [96]:
# First, import the relevant modules
import requests
import json
import numpy as np

We will now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
into the JSON structure that will be returned.

In [97]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/EON_X?start_date=2019-05-17&end_date=2019-05-17&api_key='

response = requests.get(url + API_KEY)
quandl = response.json()
quandl

{'dataset': {'id': 36592406,
  'dataset_code': 'EON_X',
  'database_code': 'FSE',
  'name': 'E.on Se (EON_X)',
  'description': 'Stock Prices for E.on Se (EON) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE000ENAG999',
  'refreshed_at': '2019-05-17T22:34:17.990Z',
  'newest_available_date': '2019-05-17',
  'oldest_available_date': '2003-01-20',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2019-05-17',
  'end_date': '2019-05-17',
  'data': [['2019-05-17',
    None,
    9.295,
    9.205,
    9.292,
    None,
    8559771.0,
    79306634.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

After inspecting the structure we can see how the file is nested.  All the keys fall into the 'dataset' key as values.  We are particularly looking at 'column_names' and 'data' as key and values pairs in a new dictionary.

In [98]:
quandl['dataset']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [99]:
quandl['dataset']['data'][0]

['2019-05-17',
 None,
 9.295,
 9.205,
 9.292,
 None,
 8559771.0,
 79306634.0,
 None,
 None,
 None]

## Analyzing the Frankfort Stock Exchange

First we will collect data from the Franfurt Stock Exchange, for the whole year 2017.  We must change the ticker to AFX_X.  We must also keep in mind some values are None values.

In [100]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key='

response = requests.get(url + API_KEY)
quandl = response.json()

We will now convert the returned JSON object into a Python dictionary.

In [101]:
fse = dict(zip(quandl['dataset']['column_names'], 
               list(zip(*quandl['dataset']['data']))))

# verify keys
fse.keys()

dict_keys(['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'])

Next, let's calculate what the highest and lowest opening prices were for the stock in 2017.

In [106]:
high = max(x for x in fse['Open'] if x != None)
low = min(x for x in fse['Open'] if x != None)

print('High: {}'.format(high))
print('Low: {}'.format(low))

High: 53.11
Low: 34.0


Now we want to see what was the largest change in any one day (based on High and Low price).

In [42]:
high = np.array(fse['High'])
low = np.array(fse['Low'])
max(high - low)

2.8100000000000023

What about the largest change between any two days (based on Closing Price)?

In [107]:
max([abs(x-y) for x, y in zip(list(fse['Close']), list(fse['Close'][1:]))])

2.559999999999995

And now, the average daily trading volume during this year?

In [60]:
sum(fse['Traded Volume']) / len(fse['Traded Volume'])

89124.33725490196

Lastly, we will find the median trading volume during this year.

In [108]:
def median(x):
    sorted_list = sorted(x)
    length = len(x)
    index = (length - 1) // 2

    if (length % 2):
        return sorted_list[index]
    else:
        return (sorted_list[index] + sorted_list[index + 1])/2
    
median(fse['Traded Volume'])

76286.0

## Conclusion

Although just a practice, API is a powerful tool for wrangling and analyzing data.  The challenge with this project was to do so using the standard Python library.